In [ ]:
'''
5. Recall that the data are dyadic.  I.e., for each crisis, we have all the pairs of countries that were 
hostile.  In our data, accordingly, we have suffixes of “1” and “2” denoting whether the column 
applies to the first or second actor.  NOTE: when we coded the data, the idea was to list the first 
country as the initiator.  NOTE: It may be the case that this choice was ambiguous.  A robustness 
test would be to change the monte carlos we’re running (see 6 and 7) to reverse the order of the 
actors.  I.e., treat actor2 as the initiator. 
 
6. First goal: try to write a monte carlo to do “historical confidence intervals”.  For each crisis, do a 
uniform draw of the different historical codings (picking ONE of them, not all) and use this in the 
sample.  Repeat for multiple samples and average the results of your model across these. 
 
7. Second goal: improve the monte carlo to pick ONE row for each crisis.  You’ll note that some 
crises have a ton of dyads; this inflates the importance of these dyads.  Ideally, for each sample 
in the monte carlo, we want 1 dyad from 1 historical understanding.  Your sample size in each of 
these samples should = the number of crises.  Give or take, your samples should have around N 
= 354. 
'''

#import libraries

import numpy as np
import pandas as pd

#DATA IMPORT, CLEANING, PREPROCESSING

df = pd.read_csv('release1.1.csv')

# drop notes column, drop complexity, drop A1 Nukes and A2 Nukes, drop name
df.drop(['complexity','crisname','ccode1', 'ccode2', 'salience', 'independence', 'a2awareness', 'a1awareness', 'a2threat', 'a1threat', 'a1intensity','a2intensity'], axis=1, inplace=True)
df.dropna(how='all', inplace=True)
#df.dropna(inplace=True)

#initialize things
first_column = 'crisno'
second_column = 'history'
unique_crisis_no = df[first_column].unique()
final_selected_rows = []

#print things
print(df)
print(unique_crisis_no)

# iterate over crisis nos
for first_value in unique_crisis_no:
    # this filters data for unique crisis no
    filtered_df_first = df[df[first_column] == first_value]
    
    # this initializes the unique history for each crisis
    unique_history = filtered_df_first[second_column].unique()
    
    if len(unique_history) > 0:
        chosen_history = np.random.choice(unique_history)
        final_filtered_df = filtered_df_first[filtered_df_first[second_column] == chosen_history] # filter data for the chosen history

    #if there are any rows to sample from; i think there always should be though
        if not final_filtered_df.empty:
            # random select
            final_selected_row = final_filtered_df.sample(n=1)
            final_selected_rows.append(final_selected_row)
        #check incase something wrong
        else:
            print('this should not be happening')

# combine to df and sort by crisno
final_combined_df = pd.concat(final_selected_rows)
final_combined_df = final_combined_df.sort_values(by=first_column)

print(final_combined_df)
output_filename = 'final_selected_rows_example.csv'
final_combined_df.to_csv(output_filename, index=False)
#model stuff starts here


In [15]:
'''
this is my monte carlo stuff but the output doesnt really make sense/ idk how to interpret it. (at least no bugs)
I pretty much just looked up python monte carlo library and tried to make it work.
lmk if you have any adivce/stuff to talk about
'''

import numpy as np
import pandas as pd
import pandas_montecarlo


#DATA IMPORT, CLEANING, PREPROCESSING

df = pd.read_csv('release1.1.csv')

# drop notes column, drop complexity, drop A1 Nukes and A2 Nukes, drop name
df.drop(['complexity','crisname','ccode1', 'ccode2', 'salience', 'independence', 'a2awareness', 'a1awareness', 'a2threat', 'a1threat', 'a1intensity','a2intensity'], axis=1, inplace=True)
df.dropna(how='all', inplace=True)
#df.dropna(inplace=True)

#initialize things
first_column = 'crisno'
second_column = 'history'
unique_crisis_no = df[first_column].unique()
final_selected_rows = []

#print things
print(df)
print(unique_crisis_no)

# iterate over crisis nos
for first_value in unique_crisis_no:
    # this filters data for unique crisis no
    filtered_df_first = df[df[first_column] == first_value]
    
    # this initializes the unique history for each crisis
    unique_history = filtered_df_first[second_column].unique()
    
    if len(unique_history) > 0:
        chosen_history = np.random.choice(unique_history)
        final_filtered_df = filtered_df_first[filtered_df_first[second_column] == chosen_history] # filter data for the chosen history

    #if there are any rows to sample from; i think there always should be though
        if not final_filtered_df.empty:
            # random select
            final_selected_row = final_filtered_df.sample(n=1)
            final_selected_rows.append(final_selected_row)
        #check incase something wrong
        else:
            print('this should not be happening')

# combine to df and sort by crisno
final_combined_df = pd.concat(final_selected_rows)
final_combined_df = final_combined_df.sort_values(by=first_column)

print(final_combined_df)
output_filename = 'final_selected_rows_example.csv'
final_combined_df.to_csv(output_filename, index=False)
#model stuff starts here

# this is for working around the categorical variables with 1hot encoding
encoded_df = pd.get_dummies(final_combined_df, columns=final_combined_df.select_dtypes(include=['object']).columns)

# monte carlo function
def monte_carlo_all_columns(df, num_simulations=1000):
    results = {}
    for col in df.columns:
        mc = df[col].montecarlo(sims=num_simulations, bust=-0.1, goal=1)
        results[col] = mc.stats
    return results

# im really not sure if this is the right library/monte carlo stuff
simulation_results = monte_carlo_all_columns(encoded_df)

# print
for col, stats in simulation_results.items():
    print(f"Results for {col}:")
    print(stats)


      crisno    year actor1 actor2  history  crisis victory1 prevail1  \
0        106  1946.0    RUS    TUR        0       1      NaN      NaN   
1        108  1946.0    UKG    RUS        0       1     True     True   
2        108  1946.0    USA    RUS        0       1     True     True   
3        108  1946.0    RUS    IRN        0       1    False    False   
4        109  1946.0    NTH    INS        0       1    False    False   
...      ...     ...    ...    ...      ...     ...      ...      ...   
1658     495  2019.0    USA    IRN        0       1     True     True   
1659     495  2019.0    USA    IRN        1       1    False     True   
1660     496  2019.0    IRN    SAU        0       1    False     True   
1661     496  2019.0    USA    IRN        0       1    False     True   
1662     496  2019.0    IRN    SAU        1       1     True     True   

     victory2 prevail2  ... ssbn_start_1 ssbn_start_2  hydrogen_start_1  \
0       False    False  ...          0.0        

In [ ]:
pip install pandas_montecarlo
